In [1]:
import os, sys, shutil, re
import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Pioneer/Pioneer-05'

In [4]:
btl_dir = 'Leg_1/ctd/'

In [5]:
os.listdir(btl_dir)

['at31005.bl',
 'at31005.hex',
 'com_port_setup_at31abc.psa',
 'at31001.sum',
 'at31007.btl',
 'AT992.XMLCON',
 'at31006.bl',
 'at31002.sum',
 'at992.bl',
 'at31003.sum',
 'at990.hex',
 'AT31008.XMLCON',
 'at31003.bl',
 'at31007.hdr',
 'at31008.btl',
 'at991.bl',
 'CTD_Summary.csv',
 'at31008.hdr',
 '09P-0749_at31abc.xmlcon',
 'AT31009.XMLCON',
 'at31006.btl',
 'at31001.bl',
 'at31003.hex',
 'AT991.XMLCON',
 'doc',
 'at991.hdr',
 'AT31002.XMLCON',
 'at31002.hex',
 'at31005.btl',
 'AT31004.XMLCON',
 'fixed_safety.dsa',
 'process',
 'Seasave_std.psa',
 'at31007.bl',
 'at31005.ros',
 'at31002.btl',
 'at31002.hdr',
 'at31006.sum',
 'at31001.hex',
 'AT31001.XMLCON',
 'main_display.dsa',
 'at991.hex',
 'at31008.sum',
 'at31003.btl',
 'AT990.XMLCON',
 'at31008.ros',
 'at31004.bl',
 'AT31005.XMLCON',
 'AT31006.XMLCON',
 'at992.hdr',
 'at31009.hdr',
 'at31006.ros',
 'at31003.hdr',
 'at31008.bl',
 'AT31003.XMLCON',
 'at31007.ros',
 'at31007.hex',
 'at31002.bl',
 'at31009.hex',
 'at990.bl',
 'AT3

In [6]:
for file in os.listdir(btl_dir):
    if file.endswith('.btl'):
        print(file)

at31007.btl
at31008.btl
at31006.btl
at31005.btl
at31002.btl
at31003.btl
at31001.btl
at31004.btl


In [9]:
with open(btl_dir+'/at31001.btl') as file:
    data = file.read()
count = 0
data_dic = {}
for line in data.splitlines():

    if line.startswith('*'):
        
        if 'filename' in line.lower():
            _, file = line.split('=')
            bottle.FileName = file.strip()
            continue
            
        elif 'nmea latitude' in line.lower():
            _, lat = line.split('=')
            bottle.Latitude = lat.strip()
            continue
            
        elif 'nmea longitude' in line.lower():
            _, lon = line.split('=')
            bottle.Longitude = lon.strip()
            continue
        
        else:
            continue
            
    elif line.startswith('#'):
        
        if 'start_time' in line.lower():
            _, datetime = line.split('=')
            datetime = re.sub('\[.*\]', '', datetime)
            bottle.StartTime = pd.to_datetime(datetime).strftime('%Y-%m-%dT%H:%M:%SZ')
            continue
        
        else:
            continue
            
    else:
        # Get rid of extraneous white space
        line = line.strip()
        
        # The first two lines will be 
            
        if line.endswith('(avg)'):
                
            # This is the first thing to do - grab the date
            dstr = re.compile('[A-Za-z]{3} [0-9]{2} [0-9]{2,4}')
            dfnd = re.search(dstr, line)
            date = pd.to_datetime(dfnd.group()).strftime('%Y-%m-%d')
            # And remove the date from the line
            line = re.sub(dstr, 'date', line)
                
            # Next, we want to split and strip the lines
            items = [x.strip() for x in line.split()]
            
            # Now, iterate over the split line and append to the dictionary
            for i,j in enumerate(data_dic.keys()):
                if items[i] == 'date':
                    data_dic[j].append(date)
                elif items[i] == '(avg)':
                    pass
                else:
                    data_dic[j].append(items[i])
                
        elif line.endswith('(sdev)'):
            # Find and pull out the time - why is this not fucking stored with the date?????
            tstr = re.compile('[0-9]{2}:[0-9]{2}:[0-9]{2}')
            tfnd = re.search(tstr, line)
            time = pd.to_datetime(tfnd.group()).strftime('%H:%M:%S')
            
            # Now add the time into the data dictionary
            data_dic['Time'].append(time)
                    
        else:
            data_dic.update({el.strip():[] for el in line.split()})
            try:
                data_dic.pop('Position')
            except:
                pass
            
            

In [10]:
df = pd.DataFrame().from_dict(data_dic)
df

,Bottle,Date,PrDM,DepSM,Latitude,Longitude,T090C,T190C,C0S/m,C1S/m,Sal00,Sal11,Sbeox0V,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0,Time
0,1,2015-10-12,443.326,439.460,39.94014,-70.88340,6.0237,6.0239,3.463375,3.463428,35.0509,35.0512,1.8863,4.4897,6.90263,0.2333,94.3332,16:39:05
1,2,2015-10-12,443.278,439.413,39.94014,-70.88340,6.0226,6.0234,3.463267,3.463380,35.0508,35.0512,1.8876,4.4967,6.90281,0.2632,93.6523,16:39:23
2,3,2015-10-12,251.878,249.798,39.94014,-70.88340,9.3260,9.3490,3.774700,3.777214,35.1869,35.1904,1.4942,2.9037,6.40054,0.1663,95.9263,16:45:09
3,4,2015-10-12,252.137,250.054,39.94014,-70.88340,9.3717,9.3705,3.779986,3.779783,35.1968,35.1959,1.5026,2.9237,6.39372,0.1657,95.9419,16:45:17
4,5,2015-10-12,52.160,51.755,39.94014,-70.88340,21.8404,21.8252,5.073489,5.071585,35.7227,35.7202,2.6380,4.7131,4.98466,0.2745,93.3688,16:55:13
5,6,2015-10-12,51.793,51.390,39.94014,-70.88340,21.8515,21.8414,5.074883,5.073466,35.7246,35.7218,2.6379,4.7141,4.98362,0.2673,93.5355,16:55:24
6,7,2015-10-12,7.836,7.776,39.94014,-70.88340,22.4018,22.4138,5.143716,5.145164,35.8277,35.8292,2.6545,4.6814,4.93237,0.2522,93.8894,16:58:51
7,8,2015-10-12,7.797,7.737,39.94014,-70.88340,22.3860,22.4075,5.141674,5.144384,35.8247,35.8283,2.6544,4.6804,4.93383,0.2515,93.9055,16:59:00
8,9,2015-10-12,7.671,7.612,39.94014,-70.88340,22.3428,22.3621,5.136259,5.138474,35.8179,35.8194,2.6520,4.6801,4.93779,0.2525,93.8834,16:59:12
9,10,2015-10-12,7.778,7.718,39.94014,-70.88340,22.3590,22.3827,5.138297,5.141054,35.8204,35.8226,2.6513,4.6744,4.93629,0.2547,93.8303,16:59:17


In [11]:
df['StartTime'] = bottle.StartTime
df['StartLat'] = bottle.Latitude
df['StartLon'] = bottle.Longitude
df['FileName'] = bottle.FileName

In [12]:
df

,Bottle,Date,PrDM,DepSM,Latitude,Longitude,T090C,T190C,C0S/m,C1S/m,...,Sbeox0V,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0,Time,StartTime,StartLat,StartLon,FileName
0,1,2015-10-12,443.326,439.460,39.94014,-70.88340,6.0237,6.0239,3.463375,3.463428,...,1.8863,4.4897,6.90263,0.2333,94.3332,16:39:05,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex
1,2,2015-10-12,443.278,439.413,39.94014,-70.88340,6.0226,6.0234,3.463267,3.463380,...,1.8876,4.4967,6.90281,0.2632,93.6523,16:39:23,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex
2,3,2015-10-12,251.878,249.798,39.94014,-70.88340,9.3260,9.3490,3.774700,3.777214,...,1.4942,2.9037,6.40054,0.1663,95.9263,16:45:09,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex
3,4,2015-10-12,252.137,250.054,39.94014,-70.88340,9.3717,9.3705,3.779986,3.779783,...,1.5026,2.9237,6.39372,0.1657,95.9419,16:45:17,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex
4,5,2015-10-12,52.160,51.755,39.94014,-70.88340,21.8404,21.8252,5.073489,5.071585,...,2.6380,4.7131,4.98466,0.2745,93.3688,16:55:13,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex
5,6,2015-10-12,51.793,51.390,39.94014,-70.88340,21.8515,21.8414,5.074883,5.073466,...,2.6379,4.7141,4.98362,0.2673,93.5355,16:55:24,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex
6,7,2015-10-12,7.836,7.776,39.94014,-70.88340,22.4018,22.4138,5.143716,5.145164,...,2.6545,4.6814,4.93237,0.2522,93.8894,16:58:51,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex
7,8,2015-10-12,7.797,7.737,39.94014,-70.88340,22.3860,22.4075,5.141674,5.144384,...,2.6544,4.6804,4.93383,0.2515,93.9055,16:59:00,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex
8,9,2015-10-12,7.671,7.612,39.94014,-70.88340,22.3428,22.3621,5.136259,5.138474,...,2.6520,4.6801,4.93779,0.2525,93.8834,16:59:12,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex
9,10,2015-10-12,7.778,7.718,39.94014,-70.88340,22.3590,22.3827,5.138297,5.141054,...,2.6513,4.6744,4.93629,0.2547,93.8303,16:59:17,2015-10-12T15:23:02Z,39 56.41 N,070 53.00 W,C:\data\ctd\at31001.hex


In [20]:
pd.to_datetime(df['Date'] + ' ' + df['Time'])

0    2015-10-12 16:39:05
1    2015-10-12 16:39:23
2    2015-10-12 16:45:09
3    2015-10-12 16:45:17
4    2015-10-12 16:55:13
5    2015-10-12 16:55:24
6    2015-10-12 16:58:51
7    2015-10-12 16:59:00
8    2015-10-12 16:59:12
9    2015-10-12 16:59:17
10   2015-10-12 16:59:23
11   2015-10-12 16:59:28
dtype: datetime64[ns]

In [ ]:
bottle.Latitude, bottle.Longitude, bottle.FileName

In [7]:
class Bottle():
    
    def __init__(self):
        
        self.FileName = ''
        self.Latitude = ''
        self.Longitude = ''
        self.StartTime = ''

In [8]:
bottle = Bottle()